In [2]:
import requests
from bs4 import BeautifulSoup
import folium
import geocoder

In [3]:
r = requests.get(r'https://www.musicotherapie-federationfrancaise.com/trouver-un-musicotherapeute/liste-des-musicotherapeutes/?')
soup = BeautifulSoup(r.text)

In [4]:
musicotherapeutes = {}
for tag in soup.find_all(['h2', 'p']):
    if tag.name == 'h2':
        name=tag.text
        musicotherapeutes[name]={}
    elif tag.text[:7] == 'Coordon':
        coordon= '\n'.join([x for x in tag if getattr(x, 'name', None) != 'br'][1:])
        musicotherapeutes[name]['Coordonnées'] = coordon      
    elif tag.text[:7] == 'Adresse':
        adresse= '\n'.join([x for x in tag if getattr(x, 'name', None) != 'br'][1:])
        musicotherapeutes[name]['Adresse'] = adresse
    elif tag.text[:7] == 'Formati':
        formation= '\n'.join([x for x in tag if getattr(x, 'name', None) != 'br'][1:])
        musicotherapeutes[name]['Formation'] = formation
    elif tag.text[:7] == 'Cadre d':
        cadre= '\n'.join([x for x in tag if getattr(x, 'name', None) != 'br'][1:])
        musicotherapeutes[name]['Cadre'] = cadre.split(' - ')


In [5]:
with requests.Session()as session:
    for name, details in musicotherapeutes.items():
        musicotherapeutes[name]["latlng"] = geocoder.osm(details["Adresse"], session=session).latlng

In [6]:
m = folium.Map([47.5,1], zoom_start=7)

In [7]:
def tagged(txt:str, tag:str):
    str = txt.strip()
    if str:
        return f"<{tag}>{txt}</{tag}>\n"
    else:
        return ""

In [8]:
for musico, details in musicotherapeutes.items():
    if details['latlng'] is not None:
        html = ""
        html += tagged(musico, 'b')
        html += tagged(details['Adresse'], 'p')
        html += tagged(details['Coordonnées'], 'p')
        html += tagged(details['Formation'], 'p')
        html += tagged(''.join(['<li>'+x for x in details['Cadre']]), 'ul')
                
        mark = folium.Marker(details['latlng'], 
                             tooltip = musico.encode('ascii', 'xmlcharrefreplace').decode(), 
                             popup = html.encode('ascii', 'xmlcharrefreplace').decode(),
                            )
        mark.add_to(m)

In [9]:
m

In [10]:
m.save('musicotherapeutes.html')
